In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

#  Import and read the charity_data.csv.
#  import pandas as pd 
#  clean_data_df = pd.read_csv("cleaned_data.csv",index_col="Unnamed: 0")
#  clean_data_df.head()


import psycopg2
import pandas as pds
from sqlalchemy import create_engine

conn = psycopg2.connect(dbname='postgres', user='postgres', password='bootcamp', host='localhost', port='5432')
clean_data_df = pds.read_sql("select * from \"final_data\"", conn);
# clean_data_df.set_index('index')
clean_data_df=clean_data_df.drop(['index'], axis=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [10]:
# define the  feaetures set
X=clean_data_df.copy()
X=X.drop("target",axis=1)
X.head()

,training_hours,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,...,company_size_500-999,company_size_1000-4999,company_size_5000-9999,company_size_10000+,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Public Sector,company_type_Pvt Ltd,company_type_Other
0,47,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,8,0,1,0,1,0,0,0,1,1,...,0,0,1,0,0,0,0,0,1,0
2,18,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,46,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,108,0,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# define target set
y=clean_data_df["target"].ravel()
y[:5]

array([0, 0, 1, 1, 0], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
#Create a StandardScaler instance
scaler=StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [16]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
cm_df = pds.DataFrame(
    cm, index=["Actually staying", "Actually leaving"], columns=["Predicted staying", "Predicted leaving"])

cm_df

,Predicted staying,Predicted leaving
Actually staying,839,28
Actually leaving,128,55


In [17]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8514285714285714

In [18]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted staying,Predicted leaving
Actually staying,839,28
Actually leaving,128,55


Accuracy Score : 0.8514285714285714
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.97      0.91       867
           1       0.66      0.30      0.41       183

    accuracy                           0.85      1050
   macro avg       0.77      0.63      0.66      1050
weighted avg       0.83      0.85      0.83      1050



In [19]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.17398746, 0.01034552, 0.01114553, 0.00196794, 0.01099205,
       0.01054756, 0.01042085, 0.00862828, 0.01235344, 0.01703505,
       0.01628682, 0.00483375, 0.00237967, 0.00377672, 0.00792278,
       0.00266164, 0.00455766, 0.01016914, 0.00313825, 0.01028841,
       0.00868949, 0.00867852, 0.00799314, 0.00618404, 0.00987492,
       0.00736789, 0.00834143, 0.00276106, 0.00654274, 0.0065954 ,
       0.00281321, 0.00904116, 0.00811218, 0.01011221, 0.01000668,
       0.00924109, 0.0074037 , 0.00601988, 0.00366145, 0.01751516,
       0.01624788, 0.01224673, 0.00701016, 0.00776647, 0.01463174,
       0.00792075, 0.28196637, 0.00924435, 0.01424263, 0.01915224,
       0.01818645, 0.011447  , 0.01400255, 0.00815456, 0.01723787,
       0.00648137, 0.00971509, 0.00841277, 0.00708496, 0.01531786,
       0.00313633])

In [32]:
# We can sort the features by their importance.
x=zip(importances,X.columns)
sorted(x,reverse=True)


[(0.28196636760068716, 'city_development_index'),
 (0.1739874648744969, 'training_hours'),
 (0.019152244799776508, 'company_size_50-99'),
 (0.018186446137087246, 'company_size_100-500'),
 (0.017515158214309965, 'experience_>20'),
 (0.017237869156363543, 'company_size_10000+'),
 (0.01703504909337337, 'education_level_Graduate'),
 (0.016286818564161695, 'education_level_Masters'),
 (0.016247881943105606, 'last_new_job_1'),
 (0.015317861465117355, 'company_type_Pvt Ltd'),
 (0.014631735075133897, 'last_new_job_>4'),
 (0.014242628372200037, 'company_size_10/49'),
 (0.014002550149004428, 'company_size_1000-4999'),
 (0.012353440607229363, 'enrolled_university_no_enrollment'),
 (0.012246726224191894, 'last_new_job_2'),
 (0.01144699720545052, 'company_size_500-999'),
 (0.011145530210517384, 'gender_Male'),
 (0.010992045109964975, 'relevent_experience_Has relevent experience'),
 (0.010547558339002231, 'relevent_experience_No relevent experience'),
 (0.010420853294262114, 'enrolled_university_Ful